# First Project

## Section Three

### Problem no.1: Design a recommendation system

#### Adding Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

#### Importing and checking the data

In [2]:
data = pd.read_csv('../Divar.csv')

C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_2080\2330768946.py:1: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Divar.csv')


In [3]:
data.head()

,Unnamed: 0,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,user_type,description,title,rent_mode,...,property_type,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius
0,0,temporary-rent,villa,karaj,mehrshahr,2024-08-01 00:00:00,مشاور املاک,۵۰۰متر\n۲۰۰متر بنا دوبلکس\n۳خواب\nاستخر آبگرم ...,باغ ویلا اجاره روزانه استخر داخل لشکرآباد سهیلیه,NaN,...,NaN,4.0,6,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,500.0
1,1,residential-sell,apartment-sell,tehran,gholhak,2024-05-01 00:00:00,مشاور املاک,دسترسی عالی به مترو و شریعتی \nمشاعات تمیز \nب...,۶۰ متر قلهک فول امکانات,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0
2,2,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,NaN,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,مقطوع,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,NaN
3,3,commercial-rent,office-rent,tehran,elahiyeh,2024-06-01 00:00:00,NaN,فرشته تاپ لوکیشن\n۹۰ متر موقعیت اداری\nیک اتاق...,فرشته ۹۰ متر دفتر کار مدرن موقعیت اداری,مقطوع,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,residential-sell,apartment-sell,mashhad,emamreza,2024-05-01 00:00:00,مشاور املاک,هلدینگ ساختمانی اکبری\n\nهمراه شما هستیم برای ...,۱۱۵ متری/شمالی رو به آفتاب/اکبری,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.shape

(1000000, 61)

#### Removing the useless Columns

In [5]:
data = data.set_index('Unnamed: 0')
data.index.name = None

In [6]:
data = data.drop(columns=['description', 'title'])
data = data.drop(columns=['user_type', 'rent_to_single'])

In [7]:
data.head()

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,rent_mode,rent_value,rent_type,price_mode,price_value,...,property_type,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius
0,temporary-rent,villa,karaj,mehrshahr,2024-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,6,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,500.0
1,residential-sell,apartment-sell,tehran,gholhak,2024-05-01 00:00:00,NaN,NaN,NaN,مقطوع,8.500000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0
2,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,مقطوع,26000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,NaN
3,commercial-rent,office-rent,tehran,elahiyeh,2024-06-01 00:00:00,مقطوع,95000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,residential-sell,apartment-sell,mashhad,emamreza,2024-05-01 00:00:00,NaN,NaN,NaN,مقطوع,5.750000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Changing the Values of room_count column to be understandable

In [8]:
data['rooms_count'] = data['rooms_count'].replace({
    'بدون اتاق': 0,
    'یک': 1,
    'دو': 2,
    'سه': 3,
    'چهار': 4,
    'پنج یا بیشتر': 5
})

C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_2080\2395350444.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['rooms_count'] = data['rooms_count'].replace({


#### Combining floor count and apt in floors for column reduction

In [9]:
floors = data['total_floors_count']
floors.fillna('0', inplace=True)
#floors.unique()
floors = floors.replace({
    '30+': 31,
    'unselect': 0   
})
floors = floors.astype('int16')

upf = data['unit_per_floor']    #Units per Floor
upf.fillna('0', inplace=True)
#upf.unique()
upf = upf.replace({
    'more_than_8': 9,
    'unselect': 0   
})
upf = upf.astype('int16')
data['total_units'] = floors * upf

In [10]:
data.head()

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,rent_mode,rent_value,rent_type,price_mode,price_value,...,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius,total_units
0,temporary-rent,villa,karaj,mehrshahr,2024-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,...,4.0,6,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,500.0,0
1,residential-sell,apartment-sell,tehran,gholhak,2024-05-01 00:00:00,NaN,NaN,NaN,مقطوع,8.500000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0,0
2,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,مقطوع,26000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,NaN,0
3,commercial-rent,office-rent,tehran,elahiyeh,2024-06-01 00:00:00,مقطوع,95000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,residential-sell,apartment-sell,mashhad,emamreza,2024-05-01 00:00:00,NaN,NaN,NaN,مقطوع,5.750000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


#### Filling the water, gas, electricity, balcony, elevator, warehouse and parking columns

In [11]:
# Correcting the has_balcony column
data.has_balcony = data.has_balcony.replace({
    'true' : True,
    'false' : False, 
    'unselect': np.nan
})
# If they had, they would have mentioned as a plus point. So we count false as default
data.has_balcony = data.has_balcony.fillna(False)

# If they had, they would have said as a plus point. So we count false as default
data.has_elevator = data.has_elevator.fillna(False)

# If they had, they would have said as a plus point. So we count false as default
data.has_warehouse = data.has_warehouse.fillna(False)

# Since it was demanded for every house to have at least one parking, default is set to true
data.has_parking = data.has_parking.fillna(True) 

# Since renovation counts as a good point, not mentioning it would count as False
data.is_rebuilt = data.is_rebuilt.fillna(False)

# it is not rational that the house wasn't mentioned that it doesn't have water, gas or electricity. So we count True as default
data.has_water = data.has_water.fillna(True)
data.has_gas = data.has_gas.fillna(True)
data.has_electricity = data.has_electricity.fillna(True)

# Since we are in Iran, we could assume any house has a single squat toilette :)
data.has_restroom = data.has_restroom.fillna('squat')

# We can mix the pool, jacuzzi and sauna and design a luxury column or even an luxury index. The missing values of the columns would be considered zero as default
data.has_jacuzzi = data.has_jacuzzi.fillna(False)
data.has_sauna = data.has_sauna.fillna(False)
data.has_pool = data.has_pool.fillna(False)
data.has_barbecue = data.has_barbecue.fillna(False)
data['luxury_index'] = data['has_pool'].astype('int') + data['has_sauna'].astype('int') + data['has_jacuzzi'].astype('int') + data['has_barbecue'].astype('int')

# Also shoofaj, water_cooler and package can be assumed as the default parameters for heating and cooling systems and warm water system relatively because it is most used (in more than have of the available data)
data.has_cooling_system = data.has_cooling_system.fillna('water_cooler')
data.has_heating_system = data.has_heating_system.fillna('shoofaj')
data.has_warm_water_provider = data.has_warm_water_provider.fillna('package')

# Most of our data is Residential so they won't have business deed by defaut
data.has_business_deed = data.has_business_deed.fillna(False)

# Most transactions only use single_page deed so we can set it as a default parameter
data.deed_type = data.deed_type.fillna('single_page')

# I assumed the 30+ floor as 31 and fill the NaN with mean+floor of the available floors
data.floor = data.floor.replace('30+', 31)
data.floor = pd.to_numeric(data.floor, errors='coerce')
data.floor = data.floor.fillna(np.floor(np.nanmean(data.floor)))

# Doing the same with room_count
data.rooms_count = data.rooms_count.fillna(np.floor(np.nanmean(data.rooms_count)))

# Binning and reforming the year column
data.construction_year = data.construction_year.replace(['قبل از ۱۳۷۰', '۱۳۷۱', '۱۳۷۲', '۱۳۷۳', '۱۳۷۴'], 1370)
data.construction_year = data.construction_year.replace(['۱۳۷۵', '۱۳۷۶', '۱۳۷۷', '۱۳۷۸', '۱۳۷۹', '۱۳۸۰', '۱۳۸۱', '۱۳۸۲', '۱۳۸۳', '۱۳۸۴'], 1380)
data.construction_year = data.construction_year.replace(['۱۳۸۵', '۱۳۸۶', '۱۳۸۷', '۱۳۸۸', '۱۳۸۹', '۱۳۹۰', '۱۳۹۱', '۱۳۹۲', '۱۳۹۳', '۱۳۹۴'], 1390)
data.construction_year = data.construction_year.replace(['۱۳۹۵', '۱۳۹۶', '۱۳۹۷', '۱۳۹۸', '۱۳۹۹', '۱۴۰۰', '۱۴۰۱', '۱۴۰۲', '۱۴۰۳'], 1400)

C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_2080\1899736385.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.has_balcony = data.has_balcony.fillna(False)
C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_2080\1899736385.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.has_elevator = data.has_elevator.fillna(False)
C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_2080\1899736385.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer

#### Deleting the unused and logically not important columns

In [12]:
data = data.drop(columns=['unit_per_floor', 'total_floors_count', 'floor_material', 'created_at_month'])
# The security guard, barbeque column has 97 percent missing value so we can safely remove them
data = data.drop(columns=['has_security_guard', 'property_type'])
# Engineered a new feature so we can delete these three columns
data = data.drop(columns=['has_jacuzzi', 'has_sauna', 'has_pool', 'has_barbecue'])

#### Trying the location algorithms

In [13]:
# Extracting all Location datas into another dataset
loc_data= data[['city_slug', 'neighborhood_slug', 'location_latitude', 'location_longitude']]

In [14]:
loc_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   city_slug           999998 non-null  object 
 1   neighborhood_slug   437139 non-null  object 
 2   location_latitude   655608 non-null  float64
 3   location_longitude  655608 non-null  float64
dtypes: float64(2), object(2)
memory usage: 38.1+ MB


In [15]:
# Filtering the Data *with* lat and long
df = loc_data[loc_data['location_latitude'].notnull() & loc_data['location_longitude'].notnull()].copy()

# Separate rows with and without neighborhood labels
df_known_neigh = df[df['neighborhood_slug'].notnull()]
df_missing_neigh = df[df['neighborhood_slug'].isnull()]

# Train a KNN classifier using lat and long as features
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(df_known_neigh[['location_latitude', 'location_longitude']], df_known_neigh['neighborhood_slug'])

# Predict and fill the missing neighborhood values
predicted = knn.predict(df_missing_neigh[['location_latitude', 'location_longitude']])

# Putting it back into loc_data
loc_data.loc[df_missing_neigh.index, 'neighborhood_slug'] = predicted

In [16]:
loc_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   city_slug           999998 non-null  object 
 1   neighborhood_slug   800249 non-null  object 
 2   location_latitude   655608 non-null  float64
 3   location_longitude  655608 non-null  float64
dtypes: float64(2), object(2)
memory usage: 70.4+ MB


In [17]:
# Designing a function to fill the neighborhood column for rows that only have city data by replacing the NaN data with most frequent neighborhood
def fill_with_frequent(group):
    if group.mode().empty:
        # Doing this to check if we have some cities that do not have any neighborhoods
        return
    else:
        return group.fillna(group.mode().iloc[0])

loc_data['neighborhood_slug'] = loc_data.groupby('city_slug')['neighborhood_slug'].transform(fill_with_frequent)

C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_2080\2499323512.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc_data['neighborhood_slug'] = loc_data.groupby('city_slug')['neighborhood_slug'].transform(fill_with_frequent)


In [18]:
loc_data[loc_data['city_slug'].isna()]

,city_slug,neighborhood_slug,location_latitude,location_longitude
292674,NaN,NaN,NaN,NaN
676502,NaN,NaN,NaN,NaN


This data has no location Data therefore can't be used in the secondary part of the question. Because of that we delete it

In [19]:
data = data.drop(loc_data[loc_data['city_slug'].isna()].index)
loc_data = loc_data.drop(loc_data[loc_data['city_slug'].isna()].index)
loc_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999998 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   city_slug           999998 non-null  object 
 1   neighborhood_slug   999998 non-null  object 
 2   location_latitude   655608 non-null  float64
 3   location_longitude  655608 non-null  float64
dtypes: float64(2), object(2)
memory usage: 38.1+ MB


### Now we fill the lat and long data based on City and Neighborhoods

In [20]:
# Calculating the center of each neighborhood for lat and long filling 
centroids = loc_data.groupby(['city_slug', 'neighborhood_slug']).agg({
    'location_latitude': 'mean',
    'location_longitude': 'mean'
}).reset_index()

# Now we merge the centres with main data
full_loc = pd.merge(loc_data, centroids, on=['city_slug', 'neighborhood_slug'], how='left', suffixes=('', '_centers'))
full_loc['location_latitude'] = full_loc['location_latitude'].fillna(full_loc['location_latitude_centers'])
full_loc['location_longitude'] = full_loc['location_longitude'].fillna(full_loc['location_longitude_centers'])

In [21]:
full_loc = full_loc.drop(columns=['location_latitude_centers', 'location_longitude_centers'])
full_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999998 entries, 0 to 999997
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   city_slug           999998 non-null  object 
 1   neighborhood_slug   999998 non-null  object 
 2   location_latitude   999994 non-null  float64
 3   location_longitude  999994 non-null  float64
dtypes: float64(2), object(2)
memory usage: 30.5+ MB


In [22]:
# We still have two neighborhoods that have no location data. Because the count is so little (4 samples) we can delete them
full_loc = full_loc.drop(full_loc[full_loc['location_latitude'].isna()].index)
data = data.drop(full_loc[full_loc['location_latitude'].isna()].index)

#### Now that we now that the algorithm works, we execute it on the main dataset

In [ ]:
# # Designing a function to fill the neighborhood column for rows that only have city data by replacing the NaN data with most frequent neighborhood
# def fill_with_frequent(group):
#     if group.mode().empty:
#         # Doing this to check if we have some cities that do not have any neighborhoods
#         return
#     else:
#         return group.fillna(group.mode().iloc[0])

# # Filtering the Data *with* lat and long
# df = data[data['location_latitude'].notnull() & data['location_longitude'].notnull()].copy()

# # Separate rows with and without neighborhood labels
# df_known_neigh = df[df['neighborhood_slug'].notnull()]
# df_missing_neigh = df[df['neighborhood_slug'].isnull()]

# # Train a KNN classifier using lat and long as features
# knn = KNeighborsClassifier(n_neighbors=5)
# knn.fit(df_known_neigh[['location_latitude', 'location_longitude']], df_known_neigh['neighborhood_slug'])

# # Predict and fill the missing neighborhood values
# predicted = knn.predict(df_missing_neigh[['location_latitude', 'location_longitude']])

# # Putting it back into loc_data
# data.loc[df_missing_neigh.index, 'neighborhood_slug'] = predicted

# data['neighborhood_slug'] = data.groupby('city_slug')['neighborhood_slug'].transform(fill_with_frequent)
# data = data.drop(data[data['city_slug'].isna()].index)      # Getting rid of data with no location

# # Calculating the center of each neighborhood for lat and long filling 
# centers = data.groupby(['city_slug', 'neighborhood_slug']).agg({
#     'location_latitude': 'mean',
#     'location_longitude': 'mean'
# }).reset_index()

# # Now we merge the centres with main data
# data = pd.merge(data, centers, on=['city_slug', 'neighborhood_slug'], how='left', suffixes=('', '_centers'))
# data['location_latitude'] = data['location_latitude'].fillna(data['location_latitude_centers'])
# data['location_longitude'] = data['location_longitude'].fillna(data['location_longitude_centers'])

# data = data.drop(columns=['location_latitude_centers', 'location_longitude_centers'])
# data = data.drop(data[data['location_latitude'].isna()].index)

### Trying Kmeans

In [ ]:
kmean = KMeans(10, random_state=42)
clusters = kmean.fit_predict(data)

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values